### Pytorch VAE example code

- [Basic VAE Example](https://github.com/pytorch/examples/blob/master/vae/main.py)

#### code를 보기전에 VAE의 특징을 알아보자. 

- VAE는 Generative model이라는 것. (training data가 주어졌을 때 이 data가 sampling된 분포와 같은 분포에서 새로운 sample을 생성하는 model)
- latent variable이라는 것이 있으며 이것을 바탕으로 데이터를 생성한다는 것(Decoder).
- 문제를 더 쉽게 만들기 위해 latent variable 이라는 것을 Encoder를 통해 추출한다는 것.
- VAE의 학습과정은 MLE라는 것.

<img src="https://www.dropbox.com/s/ce7x00eq6eltvho/Screenshot%202018-06-19%2022.05.22.png?dl=1" alt="drawing" width="800"/>

## 2. Variational Auto-Encoder

- $p_\theta(x)$ : probability density function ($\theta$라는 parameter가 주어졌을때 $x$라는 data가 나올 확률)

위 확률 밀도 함수를 **최대화**하는 것이 `Generative Model` 혹은 `density estimation`의 목표이다. 

$z$라는 **latent variable**을 사용해서 **식(1)**로 표현할 수 있다.

> $$ p_\theta(x) = \int p_\theta(z)p_\theta(x|z)dz \tag{1}\label{eq1}$$

**식(1)**을 그림으로 표현하면 다음과 같다. 

> <img src="https://www.dropbox.com/s/jyiu96dd3tp8rue/Screenshot%202018-06-20%2000.53.19.png?dl=1" alt="drawing" width="200"/> - 

즉, `VAE`는 기존 `Auto-Encoder`와 달리 latent variable($z$)을 정의하는데, 

#### Q1) 

- latent variable($z$) 필요한 이유는 무엇일까? 

#### A1) 

- 일반적으로 생성하고 싶은 데이터들은 차원이 매우 높고 datapoint 사이에 복잡한 관계가 있다. 따라서, 이러한 관계를 확률 모델로 모델링하는게 아니라 데이터를 표현하는 $z$가 있으면 그 $z$로부터 데이터를 생성하는 graphical model을 생각해보는 것이다. (**manifold 가정**)

앞으로 이 식(1) 을 미분해서 그 미분값에 따라 **stochastic gradient ascent**를 할 것입니다.

식 (1)의 구성요소는 다음과 같다.

- $p_\theta(z)$ : latent variable $z$를 sampling 할 수 있는 pdf

- $p_\theta(x|z)$ : $z$가 주어졌을 때 $x$를 생성해내는 pdf

<img src="https://www.dropbox.com/s/h0kcdfe7r0o2lqa/Screenshot%202018-06-20%2008.09.47.png?dl=1" alt="drawing" width="600"/> 

#### Q2)
- stochastic gradient ascent를 하려면 미분을 해야하는데 왜 미분이 불가능할까?

#### A2)
- 오른쪽 항을 보면, 모든 $z$에 대해 integral(적분)을 해야하는데, 모든 $z$를 구할 수 없기 때문에 integral을 할 수 없으므로, 최적의 $\theta$를 추정하기 위한 미분 역시 불가능

그럼에도 불구하고, **MLE**문제를 풀기위해서는 미분을 해야한다.

#### Q3)
- 어떻게 **MLE**를 풀 수 있을까? 우선, 미분을 위해 적분 문제를 해결할 수 있을까?

#### A3) 
- integral의 경우 integral을 다 계산하지 않고 `Monte-Carlo estimation`을 통해 estimate 할 것입니다. 아래 식으로 표현함. 대부분의 $z$에 대해서는 simple한 Gaussian 분포로부터 sampling하기 때문에 $p_\theta(x|z)$는 거의 0의 값을 가질 것이다. 따라서 sampling이 상당히 많이 필요합니다. 데이터셋이 클 경우에 이것은 너무 cost가 큽니다. 좀 더 efficient하게 이 sampling 과정을 진행하려면 data($x$)에 dependent하게 $z$를 sampling 할 필요가 있다. 여기서 **Bayesian이 등장**합니다. 

> $$ p_\theta(x) = \int p_\theta(z)p_\theta(x|z)dz \tag{1}\label{eq1} \approx \frac{1}{N} \sum_{i=1}^{N}p_\theta(x|z^{i}) $$

- 더 효율적으로 sampling하기 위해서 $x$에 dependent한 $z$를 sampling을 하는 $p_\theta(z|x)$를 생각해보는 것이다. 이는 $x$가 주어졌을 때 $x$를 생성해낼 것 같은 $z$에 대한 확률분포를 만드는 것이다. 이를 bayes'rule을 이용하면 다음과 같이 표현할 수 있다.

<img src="https://www.dropbox.com/s/fi8reyjhzjh2h46/Screenshot%202018-06-20%2008.49.00.png?dl=1" alt="drawing" width="600"/> 

- 그럼 이제, $p_\theta(z|x)$를 미분하려고 위의 식 오른쪽 항을 미분하려는데, 우리가 알고 싶어(목적으로)하는 $p_\theta(x)$를 모르기 때문에 미분이 불가능하다.

- 따라서 이 poterior를 approximate 하는 새로운 함수를 정의합니다.

- $q_\phi(z|x)$ : $\phi$라는 새로운 parameter로 표현되는 함수. (encoder 역할, 원래 poseterior를 approximate 했기 때문에 error가 존재할 것이며, 이를 고려하여 lower bound를 정의한다.)

lower bound를 고려하기 전에 VAE의 네트워크 구조를 살펴보자

<img src="https://www.dropbox.com/s/dxn7qfpfztrjduh/Screenshot%202018-06-20%2008.53.53.png?dl=1" alt="drawing" width="800"/>

**step 1** : "Encoder network"는 $q_\phi(z|x)$이며, $x$를 input으로 받아서 $z$ space 상에서 확률분포를 만든다. 이 확률 분포는 $Gaussian$으로 가정하자. 이렇게 $x$에 dependent한 $Gaussian$분포로부터 $z$를 sampling.

**step 2** : "Decoder network"는 $p_\theta(x|z)$이며, $x$의 space 상의 $Gaussian$ 또는 $Bernoulli$ 분포를 output으로 내놓는다. 그러면 $x$를 이 분포로부터 sampling할 수 있다.

즉, 위와 같은 구조를 가지기 때문에 "Auto-Encoder"가 되는 것이며 학습이 되고 나면 latent variable $z$라는 data의 의미있는  representation을 얻을 수 있습니다.

## 3. ELBO(Evidence Lower Bound)

이제 VAE를 어떻게 학습시키는지를 살펴보기 위해 objective function을 변형시켜보겠습니다. log likelihood는 다음과 같습니다. 이 값을 최대화시키는 것이 목표입니다. 이 식 자체는 intractable 하기 때문에 변형이 필요합니다.

> $$ \log p_\theta(x^{(i)})$$

위 `log likelihood`를 $q_\phi(z|x)$로부터 sampling한 latent variable $z$에 대한 기댓값으로 변경하면 아래와 같다.

> $$ \log p_\theta(x^{(i)}) = \mathbb{E}_{z~q_\phi(z|x^{(i)})} [\log p_\theta(x^{(i)})]$$

위 식에 Bayes' Rule을 적용해보자.

> $$p_\theta(z|x^{(i)}) = \frac{p_\theta(x^{(i)}|z)p_\theta(z)}{p_\theta (x^{(i)})}$$

$p_\theta (x^{(i)})$를 기준으로 정리해보자.

> $$p_\theta (x^{(i)})= \frac{p_\theta(x^{(i)}|z)p_\theta(z)}{p_\theta(z|x^{(i)})}$$

이를 `log likelihood`에 대입해보자.

> $$ \log p_\theta(x^{(i)}) = \mathbb{E}_{z~q_\phi(z|x^{(i)})} [\log \frac{p_\theta(x^{(i)}|z)p_\theta(z)}{p_\theta(z|x^{(i)})}]$$

그 다음에 expectation 안의 항에 같은 $q_\phi(z|x^{i})$를 값을 곱하고 나눕니다.

> $$ \log p_\theta(x^{(i)}) = \mathbb{E}_{z~q_\phi(z|x^{(i)})} [\log \frac{p_\theta(x^{(i)}|z)p_\theta(z)}{p_\theta(z|x^{(i)})} \times \frac{q_\phi(z|x^{(i)})}{q_\phi(z|x^{(i)})}]$$

이 때, $p_\theta(z)$와 $q_\phi(z|x^{(i)})$를 하나로 묶고 $p_\theta(x^{(i)}|z)$와 $q_\phi(z|x^{(i)})$를 하나로 묶어서 별도의 Expectation으로 내보내보자.

> $$ \log p_\theta(x^{(i)}) = \mathbb{E}_z[p_\theta(x^{(i)}|z)]\ - \ \mathbb{E}_z[\log \frac{q_\phi(z|x^{(i)})}{p_\theta(z)}] \ + \ \mathbb{E}_z [\log \frac{q_\phi(z|x^{(i)}}{p_\theta(z|x^{(i)})}]$$

우변의 두번째 항과 세번째 항은 잘 보면 **KL-Divergence**의 형태인 것을 알 수 있습니다. 따라서 KL의 형태로 바꿔쓰면 다음과 같습니다.

> $$ \log p_\theta(x^{(i)}) = \mathbb{E}_z[p_\theta(x^{(i)}|z)]\ - \ D_{KL}[q_\phi(z|x^{(i)})||p_\theta(z)] \ + \ D_{KL} [q_\phi(z|x^{(i)}||p_\theta(z|x^{(i)})]$$

- 우변의 첫번째 항 : $\mathbb{E}_z[p_\theta(x^{(i)}|z)]$ 의미 
    - $q_\phi(z|x^{(i)})$로부터 sampling한 $z$가 있으며, 그 $z$를 가지고 $p_\theta(x^{(i)}|z)$가 $x^{(i)}$를 생성할 `log likelihood` 이다.

- 우변의 두번째 항 : $D_{KL}[\log \frac{q_\phi(z|x^{(i)})}{p_\theta(z)}]$ 의미 
    - prior인 $p_\theta(z)$와 근사된 posterior인 $q_\phi(z|x^{(i)})$ 사이의 KL-divergence
    - 즉, 근사된 posterior의 분포가 얼마나 normal distribution과 가까운지에 대한 척도!

- 우변의 세번째 항 : $\mathbb{E}_z [\log \frac{q_\phi(z|x^{(i)}}{p_\theta(z|x^{(i)})}]$ 의미 
    - 원래의 posterior과 근사된 posterior의 차이로서 `approximation error`로 볼 수 있다.
    - 하지만, 앞에서 살펴봤듯이 $p_\theta(z|x^{(i)})$는 intractable 해서 세번째 항을 계산하기 어렵다. 하지만, KL 성질대로 세번째 항은 무조건 0보다 크거나 같다.

<img src="https://www.dropbox.com/s/ph4mzl3un2ai0dx/Screenshot%202018-06-20%2013.47.01.png?dl=1" alt="drawing" width="800"/>

따라서 첫번째 항과 두번째 항을 하나로 묶어주면 원래의 objective function에 대한 tractable한 lower bound를 정의할 수 있습니다. MLE 문제를 풀기 위해 objective function을 미분해서 gradient ascent 할 것입니다. Lower bound가 정의된다면 이 lower bound를 최대화하는 문제로 바꿀 수 있고 결국 lower bound의 gradient를 구하게 될 것입니다. lower bound의 두 항은 모두 미분가능하기 때문에(어떻게 미분가능한건지는 뒤에서 살펴보겠습니다) 이제 우리는 최적화를 할 수 있습니다.

<img src="https://www.dropbox.com/s/6wm7uf3nejsp21t/Screenshot%202018-06-20%2013.57.12.png?dl=1" alt="drawing" width="800"/>

lower bound를 다시 정의하자면 다음과 같습니다.

> $$L(x^{(i)}, \theta, \phi) = \mathbb{E}_z[p_\theta(x^{(i)}|z)]\ - \ D_{KL}[q_\phi(z|x^{(i)})||p_\theta(z)] $$

이 lower bound 식은 evidence의 $\log$ 값인 $p_\theta(x^{(i)}$의 lower bound이기 때문에 **Evidence Lower Bound, ELBO**라고 부릅니다.

> $$ \log p_\theta(x^{(i)}) \ge L(x^{(i)}, \theta, \phi)$$

따라서 원래 $p_\theta(x^{(i)})$를 최대화하는 문제는 다음과 같이 바뀝니다. (sampling 과정을 통해 구해지만 $x$는 모두 i.i.d로 가정

> $$ \theta^{*}, \phi^{*} = argmax_{\theta, \phi} \sum_{i=1}^{N} L(x^{(i)}, \theta, \phi) $$

지금까지 **`ELBO`**를 전개하는 과정을 정리하면 다음과 같다.

<img src="https://www.dropbox.com/s/2ro3zzf3dgo2v31/Screenshot%202018-06-20%2014.09.01.png?dl=1" alt="drawing" width="800"/>

이 ELBO를 구하는 과정은 다음 그림을 통해 이해해볼 수 있습니다. x를 encoder의 input으로 집어넣으면 encoder는 latent space 상에서의 mean과 variance를 내보냅니다(이 때, mean과 variance는 latent vector의 dimension마다 하나씩입니다). 그러면 이 mean과 variance가 posterior를 나타내게 되고 prior와의 KL을 구할 수 있습니다. 그 이후에 $z$로부터 decoder는 data의 space 상의 mean과 variance를 내보냅니다(만약 decoder의 output을 gaussian이라고 가정했다면. Bernoulli 분포라고 가정했다면 다른 형태). 그러면 ELBO의 첫번째 항 값을 구할 수 있고 ELBO가 구해집니다. 구한 값에 Backprop을 해서 업데이트하면 VAE의 학습과정이 완성됩니다.

<img src="https://www.dropbox.com/s/geesw2b5yt21bx7/Screenshot%202018-06-20%2014.10.27.png?dl=1" alt="drawing" width="800"/>

## 4. Variational Inference & Reparameterization Trick

#### VAE가 하고 싶은 것은 명확합니다. 또한 그것을 가로막는 문제도 명확히 제시합니다.

**목표**: efficient inference and learning in directed probabilistic models, in the presence of continuous latent variables  
**문제**: intractable posterior, large dataset

> $$L(x^{(i)}, \theta, \phi) = \mathbb{E}_z[p_\theta(x^{(i)}|z)]\ - \ D_{KL}[q_\phi(z|x^{(i)})||p_\theta(z)] $$

> $$ \theta^{*}, \phi^{*} = argmax_{\theta, \phi} \sum_{i=1}^{N} L(x^{(i)}, \theta, \phi) $$

위 식을 만족하는 parameter를 구하는 방법은 다음과 같다.

방법 1 : **analytic** (Mean-Field Variational Bayes)  
방법 2 : **stochastic gradient ascent**

논문에서는 방법 1을 통해 하는 경우, likelihood function인 $p_\theta(x|z)$가 NN과 같은 복잡한 함수로 표현될 경우 intractable하다고 함.  
따라서 방법 2인, gradient를 구해서 stochastic하게 parameter를 업데이트하는 방식을 사용!

#### Q4)

- $\theta$에 대한 미분은 문제 없으나, $\phi$에 대해서 미분하는 것은 불가능한것은 아니지만 왜 문제가 있다고 하는걸까? 이를 해결하기 위한 방법은? 

#### A4)

- 첫번째 항이 문제가 있다. 예를 들어 기댓값 안에 있는 함수(pdf)를 $f(z)$로 가정해보자. 이 기댓값에 대한 미분은 다음과 같이 표현할 수 있다. 

> $$\nabla_{\phi} \mathbb{E}_{q_\phi(z)}[f(z))] = \int \nabla_{\phi}q_\phi(z)f(z)dz $$

> $$ = \int q_\phi(z) \frac{\nabla_{\phi}q_\phi(z)}{q_\phi(z)}f(z)dz $$

> $$ = \mathbb{E}_{q_\phi(z)}[f(z) \nabla_{\phi} \log q_\phi(z))] $$

즉, 위에서 구한 미분값은 monte-carlo estimation을 통해 estimate 할 수 있다. 이때, $z^{(i)}$는 $q_\phi(z|x^{i})$로부터 sampling 한다.
따라서 gradient 값은 sampling 때문에 variance가 높을 것이다. 이 경우에 의해 현실적으로 미분이 어렵다는 이유라고 합니다.

> $$ \frac{1}{L} \sum_{i=1}^{L} f(z^{l}) \nabla_{\phi} \log \ q_\phi(z^{(l)})$$

- variance가 크다는 문제점을 해결하기 위해 **VAE**는 **reparameterization trick**이라는 **technique**을 사용. 즉 $z$를 posterior $q_\phi(z|x)$로부터 sampling 하는게 아니라 미분 가능한 함수 $q_\phi(\epsilon, x)$로부터 **deterministic**하게 정해진다고 보는 것이다. 이때 $\epsilon$은 noise variable이다. 

> $$\tilde z = g_\phi(\epsilon, x) \qquad where \quad \epsilon \sim p(\epsilon)$$

이 경우 다음과 같이 $p(z)$의 $q_\phi(z)$에 대한 기댓값을 $\epsilon$에 대한 기댓값으로 바꿀 수 있다.

> $$ \mathbb{E}_{q_\phi(z|x^{(i)})}[f(z)] = \mathbb{E}_{\epsilon}[f(g_\phi(\epsilon, x^{(i)}))] = \frac{1}{L}\sum_{l=1}^{L}f(g_\phi(z^{(l)}, x^{(i)})) $$

위 수식을 이용해서 **ELBO**를 고쳐쓸 수 있다. `SGVB(Stochastic Gradient Variational Bayes) estimator`라고 부름

- $z^{(i, l)}$ = $g_\phi(\epsilon^{l}, x^{(i)})$, (여기서는  $g_\phi(\epsilon, x) = \mu + \sigma\epsilon$으로 사용함(univariate gaussian case),
- $\epsilon^{(l)} \sim p(\theta)$ (헷갈림 $p(\theta)$ 맞는지 확인 필요)

> $$\tilde L ^{\mathcal{B}} (x^{(i)}, \theta, \phi) = \frac{1}{L}\sum_{l=1}^{L}f(g_\phi(x^{(i)}, z^{(i, l)}))- \ D_{KL}[q_\phi(z|x^{(i)})||p_\theta(z)] $$

이러한 `reparameterization trick`을 그림으로 보자면 다음과 같습니다. 원래는 encoder로부터 구한 data dependent한 mean과 variance를 가지고 posterior를 만듭니다. 그 posterior로부터 $z$를 샘플링한 다음에 그 $z$를 가지고 decoder는 data를 generation 했습니다. 하지만 reparametization을 하면 computation graph 내의 sampling 과정이 noise sampling이 되어 옆으로 빠져버립니다. 따라서 Back propagation을 통해 decoder output으로부터 encoder까지 gradient가 전달될 수 있습니다.

<img src="https://www.dropbox.com/s/5249ixq6r4t38l8/Screenshot%202018-06-21%2000.57.08.png?dl=1" alt="drawing" width="800"/>

이렇게 업데이트를 하는 알고리즘이 Auto-Encoding Variational Bayes이며 다음과 같습니다

<img src="https://www.dropbox.com/s/hxacd2bhz1hi3yl/Screenshot%202018-06-21%2001.05.46.png?dl=1" alt="drawing" width="800"/>

## VAE code example

**code에서 목적 함수가 가장 중요**

목적 함수를 설정하기 전에 우선 가정이 필요하다. 

- prior와 posterior를 모두 gaussian으로 가정
- likelihood를 Bernoulli라고 가정 

**`ELBO`** 식은 다음과 같이 쓸 수 있습니다. 

- 참고 : https://docs.google.com/presentation/d/175UzsMfZQ8-uuTxGO8L05KjV3iz7LOCRSAwcVVhr3-s/edit#slide=id.p36

> $$\tilde L ^{\mathcal{B}} (x^{(i)}, \theta, \phi) = \frac{1}{L}\sum_{l=1}^{L}f(g_\phi(x^{(i)}, z^{(i, l)}))- \ D_{KL}[q_\phi(z|x^{(i)})||p_\theta(z)] $$

- 오른쪽 변의 두번째 항은 다음과 같이 정리할 수 있다.
- [`Multivariate normal distributions`인 경우의 $D_{KL}$ 전개 (Example 참고)](https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence)

> $$D_{KL}[q_\phi(z|x^{(i)})||p_\theta(z)] = \frac{1}{2} \sum_{j=1}^{J}(1+\log((\alpha_j^{(i)})^2) - (\mu_j^{(i)})^2 - (\alpha_j^{(i)})^2)$$

- 오른쪽 변의 첫번째 항은 다음과 같이 정리할 수 있다.
- $y$는 $z$와 decoder를 통해 나온 값 
- 첫번째 항은 잘 보면 cross-entropy

> $$\frac{1}{L}\sum_{l=1}^{L}f(g_\phi(x^{(i)}, z^{(i, l)})) = \frac{1}{L}\sum_{l=1}^{L}((x_i \log y_{(i,l)} + (1-x_i)(1-y_{(i,l)}))$$

- 최종적으로 다음과 같이 나타낼 수 있다.

> $$\tilde L ^{\mathcal{B}} (x^{(i)}, \theta, \phi) =  \frac{1}{L}\sum_{l=1}^{L}((x_i \log y_{(i,l)} + (1-x_i)(1-y_{(i,l)})) + \frac{1}{2} \sum_{j=1}^{J}(1+\log((\alpha_j^{(i)})^2) - (\mu_j^{(i)})^2 - (\alpha_j^{(i)})^2)$$

In [97]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image


# 현재 Setup 되어있는 device 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print ('Available devices ', torch.cuda.device_count())
print ('Current cuda device ', torch.cuda.current_device())
print(torch.cuda.get_device_name(device))

Available devices  8
Current cuda device  0
GeForce GTX 1080


In [98]:
torch.manual_seed(1)

In [99]:
batch_size = 64

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True)

In [110]:
import numpy as np

In [106]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar) # variance는 0보다 크거나 같아야함, 하지만, logvar는 음의 값이 나올 수 있기 때문에 이를 양수로 만들어주는 과정
        eps = torch.randn_like(std) # noise 부분
        return mu + eps*std

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        
         # likelhood를 bern 분포로 가정했기 때문에 0~1이 나올 수 있는 sigmoid 함수 사용.
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

In [101]:
model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [102]:
# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD

In [103]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % batch_size == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))

In [104]:
def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n], recon_batch.view(batch_size, 1, 28, 28)[:n]])
                save_image(comparison.to(device), 'results/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [105]:
for epoch in range(1, 51):
    train(epoch)
    test(epoch)
    with torch.no_grad():
        sample = torch.randn(64, 20).to(device)
        sample = model.decode(sample).to(device)
        save_image(sample.view(64, 1, 28, 28),
                   'results/sample_' + str(epoch) + '.png')

Train Epoch: 1 [0/60000 (0%)]	Loss: 551.418213
Train Epoch: 1 [4096/60000 (7%)]	Loss: 197.076965
Train Epoch: 1 [8192/60000 (14%)]	Loss: 175.674347
Train Epoch: 1 [12288/60000 (20%)]	Loss: 159.718384
Train Epoch: 1 [16384/60000 (27%)]	Loss: 157.448380
Train Epoch: 1 [20480/60000 (34%)]	Loss: 144.584595
Train Epoch: 1 [24576/60000 (41%)]	Loss: 138.753418
Train Epoch: 1 [28672/60000 (48%)]	Loss: 132.026917
Train Epoch: 1 [32768/60000 (55%)]	Loss: 127.238968
Train Epoch: 1 [36864/60000 (61%)]	Loss: 125.149162
Train Epoch: 1 [40960/60000 (68%)]	Loss: 124.913773
Train Epoch: 1 [45056/60000 (75%)]	Loss: 132.296387
Train Epoch: 1 [49152/60000 (82%)]	Loss: 121.053223
Train Epoch: 1 [53248/60000 (89%)]	Loss: 124.774239
Train Epoch: 1 [57344/60000 (96%)]	Loss: 126.253571
====> Epoch: 1 Average loss: 146.9338
====> Test set loss: 119.2208
Train Epoch: 2 [0/60000 (0%)]	Loss: 116.469223
Train Epoch: 2 [4096/60000 (7%)]	Loss: 121.991882
Train Epoch: 2 [8192/60000 (14%)]	Loss: 121.196198
Train Epoch:

Train Epoch: 10 [53248/60000 (89%)]	Loss: 108.918694
Train Epoch: 10 [57344/60000 (96%)]	Loss: 106.574471
====> Epoch: 10 Average loss: 105.4424
====> Test set loss: 105.2621
Train Epoch: 11 [0/60000 (0%)]	Loss: 108.370743
Train Epoch: 11 [4096/60000 (7%)]	Loss: 99.938118
Train Epoch: 11 [8192/60000 (14%)]	Loss: 100.691666
Train Epoch: 11 [12288/60000 (20%)]	Loss: 103.675430
Train Epoch: 11 [16384/60000 (27%)]	Loss: 105.675827
Train Epoch: 11 [20480/60000 (34%)]	Loss: 109.676590
Train Epoch: 11 [24576/60000 (41%)]	Loss: 106.190811
Train Epoch: 11 [28672/60000 (48%)]	Loss: 110.429276
Train Epoch: 11 [32768/60000 (55%)]	Loss: 101.939163
Train Epoch: 11 [36864/60000 (61%)]	Loss: 104.660645
Train Epoch: 11 [40960/60000 (68%)]	Loss: 104.573120
Train Epoch: 11 [45056/60000 (75%)]	Loss: 109.492279
Train Epoch: 11 [49152/60000 (82%)]	Loss: 104.001091
Train Epoch: 11 [53248/60000 (89%)]	Loss: 104.984619
Train Epoch: 11 [57344/60000 (96%)]	Loss: 102.436752
====> Epoch: 11 Average loss: 105.2087


Train Epoch: 20 [28672/60000 (48%)]	Loss: 108.264908
Train Epoch: 20 [32768/60000 (55%)]	Loss: 103.132866
Train Epoch: 20 [36864/60000 (61%)]	Loss: 104.293991
Train Epoch: 20 [40960/60000 (68%)]	Loss: 99.161003
Train Epoch: 20 [45056/60000 (75%)]	Loss: 104.364090
Train Epoch: 20 [49152/60000 (82%)]	Loss: 106.851753
Train Epoch: 20 [53248/60000 (89%)]	Loss: 101.759102
Train Epoch: 20 [57344/60000 (96%)]	Loss: 99.735367
====> Epoch: 20 Average loss: 103.5720
====> Test set loss: 103.6074
Train Epoch: 21 [0/60000 (0%)]	Loss: 107.332169
Train Epoch: 21 [4096/60000 (7%)]	Loss: 107.285080
Train Epoch: 21 [8192/60000 (14%)]	Loss: 100.527344
Train Epoch: 21 [12288/60000 (20%)]	Loss: 102.157257
Train Epoch: 21 [16384/60000 (27%)]	Loss: 105.577484
Train Epoch: 21 [20480/60000 (34%)]	Loss: 101.020218
Train Epoch: 21 [24576/60000 (41%)]	Loss: 97.087471
Train Epoch: 21 [28672/60000 (48%)]	Loss: 101.934875
Train Epoch: 21 [32768/60000 (55%)]	Loss: 101.303162
Train Epoch: 21 [36864/60000 (61%)]	Loss:

Train Epoch: 30 [4096/60000 (7%)]	Loss: 103.586006
Train Epoch: 30 [8192/60000 (14%)]	Loss: 101.982567
Train Epoch: 30 [12288/60000 (20%)]	Loss: 90.697235
Train Epoch: 30 [16384/60000 (27%)]	Loss: 98.922165
Train Epoch: 30 [20480/60000 (34%)]	Loss: 102.408646
Train Epoch: 30 [24576/60000 (41%)]	Loss: 97.862816
Train Epoch: 30 [28672/60000 (48%)]	Loss: 98.727127
Train Epoch: 30 [32768/60000 (55%)]	Loss: 102.017685
Train Epoch: 30 [36864/60000 (61%)]	Loss: 103.044113
Train Epoch: 30 [40960/60000 (68%)]	Loss: 99.018600
Train Epoch: 30 [45056/60000 (75%)]	Loss: 104.519981
Train Epoch: 30 [49152/60000 (82%)]	Loss: 100.125694
Train Epoch: 30 [53248/60000 (89%)]	Loss: 98.638557
Train Epoch: 30 [57344/60000 (96%)]	Loss: 97.372757
====> Epoch: 30 Average loss: 102.6424
====> Test set loss: 102.6840
Train Epoch: 31 [0/60000 (0%)]	Loss: 105.473518
Train Epoch: 31 [4096/60000 (7%)]	Loss: 103.355019
Train Epoch: 31 [8192/60000 (14%)]	Loss: 106.950989
Train Epoch: 31 [12288/60000 (20%)]	Loss: 105.10

Train Epoch: 39 [45056/60000 (75%)]	Loss: 103.838333
Train Epoch: 39 [49152/60000 (82%)]	Loss: 108.141548
Train Epoch: 39 [53248/60000 (89%)]	Loss: 103.583641
Train Epoch: 39 [57344/60000 (96%)]	Loss: 104.378235
====> Epoch: 39 Average loss: 102.0648
====> Test set loss: 102.2140
Train Epoch: 40 [0/60000 (0%)]	Loss: 99.353729
Train Epoch: 40 [4096/60000 (7%)]	Loss: 101.897331
Train Epoch: 40 [8192/60000 (14%)]	Loss: 103.665031
Train Epoch: 40 [12288/60000 (20%)]	Loss: 103.305595
Train Epoch: 40 [16384/60000 (27%)]	Loss: 97.605614
Train Epoch: 40 [20480/60000 (34%)]	Loss: 106.488686
Train Epoch: 40 [24576/60000 (41%)]	Loss: 103.581192
Train Epoch: 40 [28672/60000 (48%)]	Loss: 102.906792
Train Epoch: 40 [32768/60000 (55%)]	Loss: 103.092415
Train Epoch: 40 [36864/60000 (61%)]	Loss: 101.921837
Train Epoch: 40 [40960/60000 (68%)]	Loss: 96.524292
Train Epoch: 40 [45056/60000 (75%)]	Loss: 105.983932
Train Epoch: 40 [49152/60000 (82%)]	Loss: 104.604836
Train Epoch: 40 [53248/60000 (89%)]	Loss:

Train Epoch: 49 [20480/60000 (34%)]	Loss: 102.229904
Train Epoch: 49 [24576/60000 (41%)]	Loss: 101.665688
Train Epoch: 49 [28672/60000 (48%)]	Loss: 101.288887
Train Epoch: 49 [32768/60000 (55%)]	Loss: 106.678345
Train Epoch: 49 [36864/60000 (61%)]	Loss: 97.683434
Train Epoch: 49 [40960/60000 (68%)]	Loss: 102.711014
Train Epoch: 49 [45056/60000 (75%)]	Loss: 106.213905
Train Epoch: 49 [49152/60000 (82%)]	Loss: 99.669861
Train Epoch: 49 [53248/60000 (89%)]	Loss: 103.259018
Train Epoch: 49 [57344/60000 (96%)]	Loss: 106.787827
====> Epoch: 49 Average loss: 101.6163
====> Test set loss: 101.8406
Train Epoch: 50 [0/60000 (0%)]	Loss: 99.573112
Train Epoch: 50 [4096/60000 (7%)]	Loss: 97.720924
Train Epoch: 50 [8192/60000 (14%)]	Loss: 101.502121
Train Epoch: 50 [12288/60000 (20%)]	Loss: 104.755035
Train Epoch: 50 [16384/60000 (27%)]	Loss: 100.669182
Train Epoch: 50 [20480/60000 (34%)]	Loss: 102.862839
Train Epoch: 50 [24576/60000 (41%)]	Loss: 101.821373
Train Epoch: 50 [28672/60000 (48%)]	Loss: 